# Demucs v4 - Separation de Sources Audio

**Module :** 02-Audio-Advanced  
**Niveau :** Intermediaire  
**Technologies :** Meta Demucs v4 (htdemucs_ft), ~4 GB VRAM  
**Duree estimee :** 45 minutes  

## Objectifs d'Apprentissage

- [ ] Installer et charger le modele Demucs v4 (htdemucs_ft)
- [ ] Separer un fichier audio en 4 stems : drums, bass, vocals, other
- [ ] Visualiser chaque stem (forme d'onde et spectrogramme)
- [ ] Re-mixer les stems avec des volumes differents
- [ ] Mesurer la qualite de separation (SDR - Signal-to-Distortion Ratio)
- [ ] Comparer les modeles htdemucs et htdemucs_ft
- [ ] Identifier les cas d'usage : karaoke, remixage, transcription

## Prerequis

- GPU NVIDIA avec au moins 4 GB VRAM (ou CPU)
- `pip install demucs`
- Un fichier audio pour tester (ou utiliser le fichier synthetique genere)

**Navigation :** [<< 02-3](02-3-MusicGen-Generation.ipynb) | [Index](../README.md) | [Suivant >>](../03-Orchestration/03-1-Multi-Model-Audio-Comparison.ipynb)

In [1]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres Demucs
model_name = "htdemucs_ft"           # "htdemucs" ou "htdemucs_ft" (fine-tuned)
device = "cuda"                      # "cuda" ou "cpu"
segment_size = 10                    # Taille des segments (secondes)

# Configuration
generate_audio = True              # Generer les fichiers audio de test
save_results = True                # Sauvegarder les stems
visualize_stems = True             # Visualiser les formes d'onde
compare_models = False             # Comparer htdemucs vs htdemucs_ft

In [2]:
# Setup environnement et imports
import os
import sys
import json
import time
import gc
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import logging

import numpy as np
import soundfile as sf
from IPython.display import Audio, display, HTML

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.audio_helpers import play_audio, save_audio
        print("Helpers audio importes")
    except ImportError:
        print("Helpers audio non disponibles - mode autonome")

# Repertoires
OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'audio' / 'demucs'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('demucs_separation')

# Verification GPU
gpu_available = False
try:
    import torch
    gpu_available = torch.cuda.is_available()
    if gpu_available:
        gpu_name = torch.cuda.get_device_name(0)
        gpu_vram = torch.cuda.get_device_properties(0).total_mem / (1024**3)
        print(f"GPU : {gpu_name} ({gpu_vram:.1f} GB VRAM)")
    else:
        print("GPU non disponible - Demucs fonctionne aussi sur CPU")
        if device == "cuda":
            device = "cpu"
            print("Fallback vers CPU")
except ImportError:
    print("torch non installe")
    device = "cpu"

print(f"\nDemucs v4 - Separation de Sources Audio")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}, Device : {device}")
print(f"Modele : {model_name}")
print(f"Sortie : {OUTPUT_DIR}")

Helpers audio importes


GPU non disponible - Demucs fonctionne aussi sur CPU
Fallback vers CPU

Demucs v4 - Separation de Sources Audio
Date : 2026-02-18 10:30:26
Mode : interactive, Device : cpu
Modele : htdemucs_ft
Sortie : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\demucs


In [3]:
# Chargement .env
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve")

# Demucs ne necessite pas de cle API
print(f"Demucs est un modele local - pas de cle API necessaire")

Fichier .env charge depuis : D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\.env
Demucs est un modele local - pas de cle API necessaire


## Dependances GPU OptionnellesCe notebook utilise des modeles GPU optionnels. Pour les activer:Sans ces dependances, le notebook s'executera en mode API uniquement.

## Section 1 : Presentation de Demucs

Demucs est un modele de separation de sources musicales developpe par Meta (Facebook AI Research). Il decompose un morceau en 4 stems (pistes) independantes.

### Les 4 stems

| Stem | Contenu | Exemples |
|------|---------|----------|
| `drums` | Percussions | Batterie, cymbales, hi-hat |
| `bass` | Basses | Basse electrique, contrebasse |
| `vocals` | Voix | Chant, choeurs, voix parlees |
| `other` | Autres instruments | Guitare, piano, cuivres, synthes |

### Variantes du modele

| Modele | Description | Qualite (SDR) | Vitesse |
|--------|-------------|--------------|--------|
| `htdemucs` | Hybrid Transformer Demucs | Bonne | Rapide |
| `htdemucs_ft` | Fine-tuned sur MUSDB18 | Meilleure | Rapide |
| `htdemucs_6s` | 6 sources (piano, guitare) | Variable | Plus lent |
| `mdx_extra` | MDX-Net architecture | Bonne | Rapide |

### Metriques de qualite

| Metrique | Description | Bonne valeur |
|----------|-------------|-------------|
| SDR (dB) | Signal-to-Distortion Ratio | > 7 dB |
| SIR (dB) | Signal-to-Interference Ratio | > 15 dB |
| SAR (dB) | Signal-to-Artifacts Ratio | > 5 dB |

In [4]:
# Chargement du modele Demucs
print("CHARGEMENT DU MODELE DEMUCS")
print("=" * 45)

demucs_loaded = False

try:
    from demucs.pretrained import get_model
    from demucs.apply import apply_model

    print(f"Chargement {model_name}...")
    start_time = time.time()

    separator = get_model(model_name)
    separator.to(device)
    separator.eval()
    load_time = time.time() - start_time
    demucs_loaded = True

    print(f"Modele charge en {load_time:.1f}s")
    print(f"Device : {device}")
    print(f"Sources : {separator.sources}")
    print(f"Sample rate : {separator.samplerate} Hz")
    print(f"Nombre de sources : {len(separator.sources)}")

    if gpu_available:
        vram_used = torch.cuda.memory_allocated(0) / (1024**3)
        print(f"VRAM utilisee : {vram_used:.2f} GB")

except ImportError:
    print("demucs non installe")
    print("Installation : pip install demucs")
except Exception as e:
    print(f"Erreur lors du chargement : {type(e).__name__} - {str(e)[:200]}")

CHARGEMENT DU MODELE DEMUCS
demucs non installe
Installation : pip install demucs


## Section 2 : Preparation de l'audio de test

Pour tester la separation, nous allons creer un mix synthetique compose de 4 sources controlees. Cela permet de mesurer objectivement la qualite de separation.

| Source | Signal | Description |
|--------|--------|-------------|
| Drums | Bruit pulse | Simule des frappes de batterie |
| Bass | Sinusoide grave | Simule une ligne de basse |
| Vocals | Sinusoide modulee | Simule une voix (formants) |
| Other | Accord en harmoniques | Simule un accord de guitare/piano |

In [5]:
# Creation d'un audio de test synthetique
print("CREATION DE L'AUDIO DE TEST")
print("=" * 45)

if demucs_loaded and generate_audio:
    sr = separator.samplerate  # 44100 Hz pour Demucs
    test_duration = segment_size  # secondes
    t = np.linspace(0, test_duration, int(sr * test_duration), endpoint=False)

    # Drums : impulsions rythmiques
    drums = np.zeros_like(t)
    beat_interval = int(sr * 0.5)  # 120 BPM
    for i in range(0, len(t), beat_interval):
        decay = np.exp(-30 * np.arange(min(int(sr * 0.05), len(t) - i)) / sr)
        noise_burst = np.random.randn(len(decay)) * decay
        drums[i:i+len(decay)] += noise_burst * 0.3

    # Bass : sinusoide grave (80 Hz)
    bass = 0.4 * np.sin(2 * np.pi * 80 * t) * (1 + 0.3 * np.sin(2 * np.pi * 2 * t))

    # Vocals : sinusoide modulee en frequence (simule formants)
    vocals = 0.3 * np.sin(2 * np.pi * (300 + 50 * np.sin(2 * np.pi * 3 * t)) * t)
    vocals *= (1 + 0.5 * np.sin(2 * np.pi * 1.5 * t))  # Modulation amplitude

    # Other : accord (Do majeur)
    other = (
        0.2 * np.sin(2 * np.pi * 523.25 * t) +   # C5
        0.15 * np.sin(2 * np.pi * 659.25 * t) +  # E5
        0.15 * np.sin(2 * np.pi * 783.99 * t)    # G5
    )

    # Mix stereo
    mix_mono = drums + bass + vocals + other
    mix_stereo = np.stack([mix_mono, mix_mono], axis=0)  # [2, samples]

    # Normalisation
    max_val = np.max(np.abs(mix_stereo))
    if max_val > 0:
        mix_stereo = mix_stereo / max_val * 0.9

    # Sauvegarder le mix
    mix_path = OUTPUT_DIR / "test_mix.wav"
    sf.write(str(mix_path), mix_stereo.T, sr)  # soundfile attend [samples, channels]

    print(f"Mix cree : {mix_path.name}")
    print(f"  Duree : {test_duration}s")
    print(f"  Sample rate : {sr} Hz")
    print(f"  Canaux : 2 (stereo)")
    print(f"  Sources : drums, bass, vocals, other")

    print(f"\nEcoute du mix :")
    display(Audio(data=mix_stereo, rate=sr))

    # Sauvegarder aussi les sources individuelles (ground truth)
    ground_truth = {"drums": drums, "bass": bass, "vocals": vocals, "other": other}
    for name, source in ground_truth.items():
        gt_path = OUTPUT_DIR / f"gt_{name}.wav"
        source_stereo = np.stack([source, source], axis=0)
        sf.write(str(gt_path), source_stereo.T, sr)
else:
    print("Modele non charge ou generation desactivee")

CREATION DE L'AUDIO DE TEST
Modele non charge ou generation desactivee


## Section 3 : Separation en 4 stems

Demucs separe l'audio en traitant le signal par segments. La fonction `apply_model` gere automatiquement le decoupage et la reconstruction.

| Parametre | Description |
|-----------|-------------|
| `shifts` | Nombre de decalages temporels pour ameliorer la qualite |
| `overlap` | Chevauchement entre segments (0.0-0.5) |
| `segment` | Taille du segment en secondes |

In [6]:
# Separation en 4 stems
print("SEPARATION EN 4 STEMS")
print("=" * 45)

stem_results = {}

if demucs_loaded and generate_audio:
    mix_path = OUTPUT_DIR / "test_mix.wav"

    if mix_path.exists():
        # Charger le mix
        import torchaudio
        mix_tensor, sr = torchaudio.load(str(mix_path))
        mix_tensor = mix_tensor.to(device)

        # Ajouter dimension batch [batch, channels, samples]
        mix_batch = mix_tensor.unsqueeze(0)

        print(f"Audio charge : {mix_tensor.shape}, {sr} Hz")
        print(f"Separation en cours...")

        start_time = time.time()
        with torch.no_grad():
            sources = apply_model(
                separator,
                mix_batch,
                shifts=1,
                overlap=0.25
            )
        sep_time = time.time() - start_time

        # sources shape: [batch, sources, channels, samples]
        print(f"\nSeparation terminee en {sep_time:.2f}s")
        print(f"Shape des sources : {sources.shape}")

        # Extraire et afficher chaque stem
        source_names = separator.sources
        for i, name in enumerate(source_names):
            stem = sources[0, i].cpu().numpy()  # [channels, samples]
            stem_duration = stem.shape[1] / sr
            stem_rms = np.sqrt(np.mean(stem**2))

            stem_results[name] = {
                "rms": stem_rms,
                "peak": np.max(np.abs(stem)),
                "samples": stem
            }

            print(f"\n  {name.upper()} :")
            print(f"    RMS : {stem_rms:.4f} | Peak : {np.max(np.abs(stem)):.4f}")
            display(Audio(data=stem, rate=sr))

            if save_results:
                stem_path = OUTPUT_DIR / f"stem_{name}.wav"
                sf.write(str(stem_path), stem.T, sr)

        # Tableau recapitulatif
        print(f"\nRecapitulatif de la separation :")
        print(f"{'Source':<12} {'RMS':<10} {'Peak':<10}")
        print("-" * 32)
        for name, data in stem_results.items():
            print(f"{name:<12} {data['rms']:<10.4f} {data['peak']:<10.4f}")
        print(f"\nTemps de separation : {sep_time:.2f}s")
    else:
        print("Fichier mix non trouve")
else:
    print("Modele non charge ou generation desactivee")

SEPARATION EN 4 STEMS
Modele non charge ou generation desactivee


### Interpretation : Separation en 4 stems

| Stem | Observation | Qualite typique |
|------|-------------|----------------|
| Drums | Bien isole, peu de bleed | SDR > 8 dB |
| Bass | Bonne separation | SDR > 7 dB |
| Vocals | Excellente separation | SDR > 9 dB |
| Other | Variable selon le contenu | SDR > 6 dB |

**Points cles** :
1. Les voix sont generalement la source la mieux separee
2. La basse et les drums ont des frequences distinctes, facilitant la separation
3. "Other" est la categorie fourre-tout, la qualite depend de la complexite du mix

## Section 4 : Visualisation et remixage

Apres la separation, on peut visualiser chaque stem et re-mixer les sources avec des volumes differents.

### Cas d'usage du remixage

| Cas d'usage | Configuration |
|-------------|---------------|
| Karaoke | Mute vocals, garder le reste |
| Isolation voix | Solo vocals, mute le reste |
| Boost basse | Augmenter bass +3dB |
| Sans batterie | Mute drums |

In [7]:
# Visualisation et remixage des stems
print("VISUALISATION ET REMIXAGE")
print("=" * 45)

if stem_results and visualize_stems:
    try:
        import matplotlib.pyplot as plt

        fig, axes = plt.subplots(len(stem_results) + 1, 1, figsize=(14, 3 * (len(stem_results) + 1)))

        # Afficher le mix original
        mix_data, mix_sr = sf.read(str(OUTPUT_DIR / "test_mix.wav"))
        t_axis = np.arange(len(mix_data)) / mix_sr
        axes[0].plot(t_axis, mix_data[:, 0], color='gray', linewidth=0.5)
        axes[0].set_title('Mix original', fontsize=12, fontweight='bold')
        axes[0].set_ylabel('Amplitude')
        axes[0].set_xlim(0, t_axis[-1])

        # Afficher chaque stem
        colors = ['#e74c3c', '#2ecc71', '#3498db', '#f39c12']
        for idx, (name, data) in enumerate(stem_results.items()):
            stem = data['samples'][0]  # Canal gauche
            t_stem = np.arange(len(stem)) / sr
            axes[idx + 1].plot(t_stem, stem, color=colors[idx % len(colors)], linewidth=0.5)
            axes[idx + 1].set_title(f'{name.upper()} (RMS: {data["rms"]:.4f})', fontsize=12, fontweight='bold')
            axes[idx + 1].set_ylabel('Amplitude')
            axes[idx + 1].set_xlim(0, t_stem[-1])

        axes[-1].set_xlabel('Temps (s)')
        plt.tight_layout()
        plt.savefig(str(OUTPUT_DIR / "stems_waveforms.png"), dpi=100, bbox_inches='tight')
        plt.show()
        print(f"Visualisation sauvegardee : stems_waveforms.png")

    except ImportError:
        print("matplotlib non disponible pour la visualisation")

    # --- Remixage ---
    print(f"\n--- REMIXAGE ---")

    remix_configs = {
        "Karaoke (sans voix)": {"drums": 1.0, "bass": 1.0, "vocals": 0.0, "other": 1.0},
        "Voix solo": {"drums": 0.0, "bass": 0.0, "vocals": 1.0, "other": 0.0},
        "Bass boost (+6dB)": {"drums": 1.0, "bass": 2.0, "vocals": 1.0, "other": 1.0},
        "Sans batterie": {"drums": 0.0, "bass": 1.0, "vocals": 1.0, "other": 1.0},
    }

    for config_name, volumes in remix_configs.items():
        print(f"\n  {config_name} :")
        print(f"    Volumes : {volumes}")

        remix = np.zeros_like(list(stem_results.values())[0]['samples'])
        for name, vol in volumes.items():
            if name in stem_results:
                remix += stem_results[name]['samples'] * vol

        # Normalisation pour eviter le clipping
        max_val = np.max(np.abs(remix))
        if max_val > 1.0:
            remix = remix / max_val * 0.95

        display(Audio(data=remix, rate=sr))

        if save_results:
            remix_path = OUTPUT_DIR / f"remix_{config_name.lower().replace(' ', '_').replace('(', '').replace(')', '')}.wav"
            sf.write(str(remix_path), remix.T, sr)
else:
    print("Stems non disponibles pour la visualisation")

VISUALISATION ET REMIXAGE
Stems non disponibles pour la visualisation


### Interpretation : Visualisation et remixage

| Remix | Qualite | Cas d'usage |
|-------|---------|-------------|
| Karaoke | Bonne si voix bien separee | Soirees, pratique chant |
| Voix solo | Tres bonne | Transcription, analyse |
| Bass boost | Excellente | DJing, mastering |
| Sans batterie | Bonne | Pratique instrument, sampling |

**Points cles** :
1. La qualite du remixage depend directement de la qualite de separation
2. Le mode karaoke est l'usage le plus populaire de Demucs
3. Le boost de stems peut creer du clipping - normaliser apres remixage

In [8]:
# Mode interactif - Separation personnalisee
if notebook_mode == "interactive" and not skip_widgets:
    print("MODE INTERACTIF - SEPARATION PERSONNALISEE")
    print("=" * 50)
    print("\nFournissez le chemin vers un fichier audio a separer :")
    print("(Laissez vide pour passer a la suite)")
    print("Formats acceptes : WAV, MP3, FLAC")

    try:
        user_path = input("\nChemin du fichier : ")

        if user_path.strip() and demucs_loaded:
            user_file = Path(user_path.strip())
            if user_file.exists():
                print(f"\nSeparation de : {user_file.name}")

                import torchaudio
                user_audio, user_sr = torchaudio.load(str(user_file))

                # Resample si necessaire
                if user_sr != separator.samplerate:
                    resampler = torchaudio.transforms.Resample(user_sr, separator.samplerate)
                    user_audio = resampler(user_audio)

                user_audio = user_audio.to(device).unsqueeze(0)

                start_time = time.time()
                with torch.no_grad():
                    user_sources = apply_model(separator, user_audio, shifts=1, overlap=0.25)
                sep_time = time.time() - start_time

                print(f"Separation terminee en {sep_time:.2f}s")

                for i, name in enumerate(separator.sources):
                    stem = user_sources[0, i].cpu().numpy()
                    print(f"\n  {name.upper()} :")
                    display(Audio(data=stem, rate=separator.samplerate))

                    if save_results:
                        stem_path = OUTPUT_DIR / f"user_{name}.wav"
                        sf.write(str(stem_path), stem.T, separator.samplerate)
            else:
                print(f"Fichier non trouve : {user_file}")
        else:
            print("Mode interactif ignore")

    except (KeyboardInterrupt, EOFError):
        print("Mode interactif interrompu")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("Mode interactif non disponible (execution automatisee)")
        else:
            print(f"Erreur : {error_type} - {str(e)[:100]}")
else:
    print("Mode batch - Interface interactive desactivee")

MODE INTERACTIF - SEPARATION PERSONNALISEE

Fournissez le chemin vers un fichier audio a separer :
(Laissez vide pour passer a la suite)
Formats acceptes : WAV, MP3, FLAC
Mode interactif non disponible (execution automatisee)


## Bonnes pratiques et cas d'usage

### Optimisation de la qualite

| Technique | Impact | Description |
|-----------|--------|-------------|
| `shifts=2` | Qualite +5-10% | Augmente le nombre de decalages (plus lent) |
| Audio haute qualite | Qualite ++ | Utiliser WAV/FLAC plutot que MP3 compresse |
| Segment adapte | Qualite variable | Ajuster `segment` selon la memoire disponible |
| `htdemucs_ft` | Qualite +2-3 dB SDR | Modele fine-tune, meilleur que htdemucs |

### Applications industrielles

| Application | Description | Stems utilises |
|-------------|-------------|----------------|
| Karaoke | Suppression de voix | Tous sauf vocals |
| Transcription | Isolation de la parole | Vocals |
| Remasterisation | Re-equilibrage du mix | Tous |
| DJ / Remix | Reutilisation de elements | Variable |
| Analyse musicale | Etude de la structure | Tous |

In [9]:
# Statistiques de session et prochaines etapes
print("STATISTIQUES DE SESSION")
print("=" * 45)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Modele : {model_name}")
print(f"Device : {device}")
print(f"Modele charge : {'Oui' if demucs_loaded else 'Non'}")

if gpu_available:
    vram_current = torch.cuda.memory_allocated(0) / (1024**3)
    print(f"VRAM utilisee : {vram_current:.2f} GB")

if save_results:
    saved = list(OUTPUT_DIR.glob('*'))
    total_size = sum(f.stat().st_size for f in saved if f.is_file()) / (1024*1024)
    print(f"Fichiers sauvegardes : {len(saved)} ({total_size:.1f} MB) dans {OUTPUT_DIR}")

if stem_results:
    print(f"\nResume des stems separes :")
    for name, data in stem_results.items():
        print(f"  {name:<10} : RMS={data['rms']:.4f}, Peak={data['peak']:.4f}")

# Liberation memoire
if demucs_loaded:
    print(f"\nLiberation du modele...")
    del separator
    gc.collect()
    if gpu_available:
        torch.cuda.empty_cache()
    print(f"Memoire liberee")

print(f"\nPROCHAINES ETAPES")
print(f"1. Comparer tous les modeles audio (03-1)")
print(f"2. Construire un pipeline vocal complet (03-2)")
print(f"3. Explorer l'API Realtime d'OpenAI (03-3)")
print(f"4. Creer des compositions multi-etapes (04-3)")

print(f"\nNotebook Demucs Source Separation termine - {datetime.now().strftime('%H:%M:%S')}")

STATISTIQUES DE SESSION
Date : 2026-02-18 10:30:26
Modele : htdemucs_ft
Device : cpu
Modele charge : Non
Fichiers sauvegardes : 14 (22.1 MB) dans D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\audio\demucs

PROCHAINES ETAPES
1. Comparer tous les modeles audio (03-1)
2. Construire un pipeline vocal complet (03-2)
3. Explorer l'API Realtime d'OpenAI (03-3)
4. Creer des compositions multi-etapes (04-3)

Notebook Demucs Source Separation termine - 10:30:26
